In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

In [7]:
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name)

print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")
   
# check if GPU is available or not, if it is mount model on GPU
# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device(device)
print(f"Using device: {device}")

model.to(device)

# create input tensor on CPU
input_ids = torch.tensor([[1, 2, 3], [4, 5, 6]])

Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [5]:
#using tokenzier.encode/decode() and model.generate()
def generate_response(input):

    tokenized_input = tokenizer.encode(input, return_tensors="pt")
    
  # Create an attention mask
    attention_mask = torch.ones(tokenized_input.shape)

    tokenized_output = model.generate(
        tokenized_input,
        max_length=150,
        num_return_sequences=1,
        attention_mask=attention_mask,  # Pass the attention mask
        pad_token_id=tokenizer.eos_token_id  # Set pad_token_id
    )
    
    return tokenizer.decode(tokenized_output[0])
    
def generate_response_using_pipeline(input):
    generator = pipeline("text-generation", model="gpt2")
    response = generator(input, max_length=100, num_return_sequences=1)
    return response[0]["generated_text"]

In [6]:
msg ="Really liked this movie!"

print(generate_response(msg))

/Users/jayreddy/prompt_eng/prompt-eng-sandbox/venv/lib/python3.11/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


RuntimeError: Placeholder storage has not been allocated on MPS device!